In [1]:
from Dataset.Dataset import CSVLoader
from compoundFeaturization.rdkitFingerprints import MorganFingerprint
from featureSelection.baseFeatureSelector import LowVarianceFS
from splitters.splitters import SingletaskStratifiedSplitter
from models.sklearnModels import SklearnModel
from metrics.Metrics import Metric
from metrics.metricsFunctions import roc_auc_score, precision_score, accuracy_score, confusion_matrix, classification_report
from parameterOptimization.HyperparameterOpt import GridHyperparamOpt

from sklearn.ensemble import RandomForestClassifier

In [2]:
#Load Dataset
dataset = CSVLoader('preprocessed_dataset_wfoodb.csv', 'Smiles', ['Class'], 'ID')#, chunk_size=2000)
print(dataset.get_shape())

(23290,) (23290,) (0,) (23290,)
((23290,), (23290,), (0,), (23290,))


In [3]:
#Featurization
dataset = MorganFingerprint().featurize(dataset)
dataset.get_shape()

Featurizing datapoint 0
Featurizing datapoint 1000
Featurizing datapoint 2000
Featurizing datapoint 3000
Featurizing datapoint 4000
Featurizing datapoint 5000
Featurizing datapoint 6000


RDKit ERROR: [16:31:59] Explicit valence for atom # 1 Cl, 4, is greater than permitted


error in smile: O=[Cl]=O
Featurizing datapoint 7000


RDKit ERROR: [16:32:03] Explicit valence for atom # 3 B, 4, is greater than permitted
RDKit ERROR: [16:32:03] Explicit valence for atom # 1 Cl, 9, is greater than permitted


error in smile: OB1O[B]2(O)OB(O)O[B](O)(O1)O2
error in smile: O=[Cl-](=O)(=O)=O
Featurizing datapoint 8000
Featurizing datapoint 9000
Featurizing datapoint 10000
Featurizing datapoint 11000
Featurizing datapoint 12000
Featurizing datapoint 13000
Featurizing datapoint 14000


RDKit ERROR: [16:32:17] Explicit valence for atom # 0 P, 11, is greater than permitted


error in smile: [P](OCC=C(C)C)(OCC=C(C)C)(=O)(OP(OCC=C(C)C)(OCC=C(C)C)=O)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)CC=C(C)C
Featurizing datapoint 15000
Featurizing datapoint 16000
Featurizing datapoint 17000
Featurizing datapoint 18000
Featurizing datapoint 19000
Featurizing datapoint 20000
Featurizing datapoint 21000
Featurizing datapoint 22000
Featurizing datapoint 23000
Elements with indexes:  [6257, 7708, 7709, 14244]  were removed due to the presence of NAs!
The elements in question are:  ['O=[Cl]=O' 'OB1O[B]2(O)OB(O)O[B](O)(O1)O2' 'O=[Cl-](=O)(=O)=O'
 '[P](OCC=C(C)C)(OCC=C(C)C)(=O)(OP(OCC=C(C)C)(OCC=C(C)C)=O)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)CC=C(C)C']
(23286,) (23286,) (23286, 1024) (23286,)


((23286,), (23286,), (23286, 1024), (23286,))

In [4]:
#Feature Selection
dataset = LowVarianceFS(0.15).featureSelection(dataset)
dataset.get_shape()

(23286,) (23286,) (23286, 49) (23286,)


((23286,), (23286,), (23286, 49), (23286,))

In [5]:
#Data Split
splitter = SingletaskStratifiedSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset=dataset, frac_train=0.6, 
                                                                             frac_valid=0.2, frac_test=0.2)

Computing train/valid/test indices


In [6]:
#Scikit-Learn Random Forest
rf = RandomForestClassifier()
model = SklearnModel(model=rf)

In [7]:
#cross validation
model.cross_validate(dataset, Metric(roc_auc_score), folds=3)

Computing Stratified K-fold split
Train Score: 
roc_auc_score: 
 0.8491654641185346
Test Score: 
roc_auc_score: 
 0.6476046658886047
Train Score: 
roc_auc_score: 
 0.8648190924324307
Test Score: 
roc_auc_score: 
 0.6515044937977013
Train Score: 
roc_auc_score: 
 0.8597454064316816
Test Score: 
roc_auc_score: 
 0.6416691522941644


(SklearnModel(model=RandomForestClassifier(), model_dir='/tmp/tmpek562mul'),
 0.8648190924324307,
 0.6515044937977013,
 [0.8491654641185346, 0.8648190924324307, 0.8597454064316816],
 [0.6476046658886047, 0.6515044937977013, 0.6416691522941644],
 0.8579099876608822,
 0.6469261039934902)

In [8]:
# model training
model.fit(train_dataset)

In [9]:
metrics = [Metric(roc_auc_score), Metric(precision_score), Metric(accuracy_score), Metric(confusion_matrix), 
           Metric(classification_report)]
print("#############################")
# evaluate the model
print('Training Dataset: ')
train_score = model.evaluate(train_dataset, metrics)
print("#############################")
print('Validation Dataset: ')
valid_score = model.evaluate(valid_dataset, metrics)
print("#############################")
print('Test Dataset: ')
test_score = model.evaluate(test_dataset, metrics)
print("#############################")

#############################
Training Dataset: 
roc_auc_score: 
 0.864409619250826
precision_score: 
 0.927652733118971
accuracy_score: 
 0.981672394043528
confusion_matrix: 
 [[13135    45]
 [  211   577]]
classification_report: 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99     13180
           1       0.93      0.73      0.82       788

    accuracy                           0.98     13968
   macro avg       0.96      0.86      0.90     13968
weighted avg       0.98      0.98      0.98     13968

ERROR in task averager:  cannot perform reduce with flexible type
#############################
Validation Dataset: 
roc_auc_score: 
 0.6390027322404372
precision_score: 
 0.5620437956204379
accuracy_score: 
 0.9469501718213058
confusion_matrix: 
 [[4332   60]
 [ 187   77]]
classification_report: 
               precision    recall  f1-score   support

           0       0.96      0.99      0.97      4392
           1       0.56      0.

In [11]:
#Build a model function for hyperparameter optimization
def rf_model_builder(n_estimators, max_features, class_weight, model_dir=None):
    rf_model = RandomForestClassifier(n_estimators=n_estimators, max_features=max_features, 
                                      class_weight=class_weight)
    return SklearnModel(rf_model, model_dir)

params_dict_rf = {"n_estimators": [10, 100],
                  "max_features": ["auto", "sqrt", "log2", None],
                  "class_weight": [{0: 1., 1: 1.}, {0: 1., 1: 5}, {0: 1., 1: 10}]
                  }
    


In [12]:
#Hyperparameter Optimization
optimizer = GridHyperparamOpt(rf_model_builder)

best_rf, best_hyperparams, all_results = optimizer.hyperparam_search(params_dict_rf, train_dataset, valid_dataset, Metric(roc_auc_score))

print('#################')
print(best_hyperparams)
print(best_rf)

Fitting 15 random models from a space of 24 possible models.
Fitting model 1/15
hyperparameters: {'n_estimators': 10, 'max_features': 'auto', 'class_weight': {0: 1.0, 1: 1.0}}
roc_auc_score: 
 0.6488450074515648
Model 1/15, Metric roc_auc_score, Validation set 1: 0.648845
	best_validation_score so far: 0.648845
Fitting model 2/15
hyperparameters: {'n_estimators': 10, 'max_features': 'auto', 'class_weight': {0: 1.0, 1: 5}}
roc_auc_score: 
 0.6604259811227025
Model 2/15, Metric roc_auc_score, Validation set 2: 0.660426
	best_validation_score so far: 0.660426
Fitting model 3/15
hyperparameters: {'n_estimators': 10, 'max_features': 'auto', 'class_weight': {0: 1.0, 1: 10}}
roc_auc_score: 
 0.6854197714853453
Model 3/15, Metric roc_auc_score, Validation set 3: 0.685420
	best_validation_score so far: 0.685420
Fitting model 4/15
hyperparameters: {'n_estimators': 10, 'max_features': 'sqrt', 'class_weight': {0: 1.0, 1: 10}}
roc_auc_score: 
 0.6750393277032621
Model 4/15, Metric roc_auc_score, Va

In [13]:
#Evaluate model
best_rf.evaluate(test_dataset, metrics)

roc_auc_score: 
 0.6781751945686372
precision_score: 
 0.33229813664596275
accuracy_score: 
 0.9201030927835051
confusion_matrix: 
 [[4177  215]
 [ 157  107]]
classification_report: 
               precision    recall  f1-score   support

           0       0.96      0.95      0.96      4392
           1       0.33      0.41      0.37       264

    accuracy                           0.92      4656
   macro avg       0.65      0.68      0.66      4656
weighted avg       0.93      0.92      0.92      4656

ERROR in task averager:  cannot perform reduce with flexible type


{'roc_auc_score': 0.6781751945686372,
 'precision_score': 0.33229813664596275,
 'accuracy_score': 0.9201030927835051,
 'confusion_matrix': 1164.0,
 'classification_report': None}